In [4]:
from pyspark import SparkContext, SparkConf

sc.stop()
conf = SparkConf().setMaster("local[2]").setAppName("Practice")
conf.set("spark.driver.cores", "1")
conf.set("spark.driver.memory", "1g")
conf.set("spark.executor.memory","1g")

sc = SparkContext(conf = conf)

In [1]:
# Extracting data from the textFile
sourcePath = "/Users/pravinkumar/Documents/Spark/testData/cards/"
textRDD = sc.textFile(sourcePath + "deckofcards.txt").map(lambda rec: (None, rec))
for i in textRDD.take(10): print(i)

(None, 'BLACK|SPADE|2')
(None, 'BLACK|SPADE|3')
(None, 'BLACK|SPADE|4')
(None, 'BLACK|SPADE|5')
(None, 'BLACK|SPADE|6')
(None, 'BLACK|SPADE|7')
(None, 'BLACK|SPADE|8')
(None, 'BLACK|SPADE|9')
(None, 'BLACK|SPADE|10')
(None, 'BLACK|SPADE|J')


In [2]:
Destinationpath = "/Users/pravinkumar/Documents/Spark/Results/Practice/"

textRDD.repartition(1).saveAsTextFile(Destinationpath + "DeckTextComp", "org.apache.hadoop.io.compress.DefaultCodec")
textRDD.repartition(1).saveAsSequenceFile(Destinationpath + "DeckSeqComp", "org.apache.hadoop.io.compress.DefaultCodec")

In [6]:
seqDeckRDD = sc.sequenceFile("/Users/pravinkumar/Documents/Spark/Results/Practice/DeckSeqComp", "org.apache.hadoop.io.Text", "org.apache.hadoop.io.Text")
for i in seqDeckRDD.take(10): print(i)
print("*"*75)
textDeckRDD = sc.textFile("/Users/pravinkumar/Documents/Spark/Results/Practice/DeckTextComp")
for i in textDeckRDD.take(10): print(i)

(None, 'BLACK|SPADE|2')
(None, 'BLACK|SPADE|3')
(None, 'BLACK|SPADE|4')
(None, 'BLACK|SPADE|5')
(None, 'BLACK|SPADE|6')
(None, 'BLACK|SPADE|7')
(None, 'BLACK|SPADE|8')
(None, 'BLACK|SPADE|9')
(None, 'BLACK|SPADE|10')
(None, 'BLACK|SPADE|J')
***************************************************************************
(None, 'BLACK|SPADE|2')
(None, 'BLACK|SPADE|3')
(None, 'BLACK|SPADE|4')
(None, 'BLACK|SPADE|5')
(None, 'BLACK|SPADE|6')
(None, 'BLACK|SPADE|7')
(None, 'BLACK|SPADE|8')
(None, 'BLACK|SPADE|9')
(None, 'BLACK|SPADE|10')
(None, 'BLACK|SPADE|J')


In [15]:
# RDD to DF
from pyspark.sql import Row
textDF = sc.textFile(sourcePath + "deckofcards.txt").map(lambda rec: rec.split("|")).\
map(lambda rec: Row(color = rec[0], shade = rec[1], symbol = rec[2])).toDF()

textDF.show()

+-----+-----+------+
|color|shade|symbol|
+-----+-----+------+
|BLACK|SPADE|     2|
|BLACK|SPADE|     3|
|BLACK|SPADE|     4|
|BLACK|SPADE|     5|
|BLACK|SPADE|     6|
|BLACK|SPADE|     7|
|BLACK|SPADE|     8|
|BLACK|SPADE|     9|
|BLACK|SPADE|    10|
|BLACK|SPADE|     J|
|BLACK|SPADE|     Q|
|BLACK|SPADE|     K|
|BLACK|SPADE|     A|
|BLACK| CLUB|     2|
|BLACK| CLUB|     3|
|BLACK| CLUB|     4|
|BLACK| CLUB|     5|
|BLACK| CLUB|     6|
|BLACK| CLUB|     7|
|BLACK| CLUB|     8|
+-----+-----+------+
only showing top 20 rows



In [28]:
textDF.repartition(1).write.csv(Destinationpath + "DeckCSVComp", compression = "org.apache.hadoop.io.compress.DefaultCodec")

sqlContext.setConf("spark.sql.parquet.compression.codec", "snappy")
textDF.repartition(1).write.parquet(Destinationpath + "DeckParqComp")

sqlContext.setConf("spark.sql.orc.compression.codec", "snappy")
textDF.repartition(1).write.orc(Destinationpath + "DeckORCComp")

textDF.repartition(1).write.json(Destinationpath + "DeckJSONComp", compression = "org.apache.hadoop.io.compress.DefaultCodec")
textDF.repartition(1).toJSON().saveAsTextFile(Destinationpath + "DeckTOJSONComp", "org.apache.hadoop.io.compress.DefaultCodec")



In [34]:
DFFromCSV = sqlContext.read.csv(Destinationpath + "DeckCSVComp")
DFFromCSV.limit(2).show()
DFFromParq = sqlContext.read.parquet(Destinationpath + "DeckParqComp")
DFFromParq.limit(2).show()
DFFromORC = sqlContext.read.orc(Destinationpath + "DeckORCComp")
DFFromORC.limit(2).show()
DFFromJson = sqlContext.read.json(Destinationpath + "DeckJSONComp")
DFFromJson.limit(2).show()

+-----+-----+---+
|  _c0|  _c1|_c2|
+-----+-----+---+
|BLACK|SPADE|  2|
|BLACK|SPADE|  3|
+-----+-----+---+

+-----+-----+------+
|color|shade|symbol|
+-----+-----+------+
|BLACK|SPADE|     2|
|BLACK|SPADE|     3|
+-----+-----+------+

+-----+-----+------+
|color|shade|symbol|
+-----+-----+------+
|BLACK|SPADE|     2|
|BLACK|SPADE|     3|
+-----+-----+------+

+-----+-----+------+
|color|shade|symbol|
+-----+-----+------+
|BLACK|SPADE|     2|
|BLACK|SPADE|     3|
+-----+-----+------+



In [44]:
# WOrdcount
wordCountReduceByRDD = sc.textFile(sourcePath + "deckofcards.txt").map(lambda rec: rec.split("|")).flatMap(lambda rec: rec).\
map(lambda rec: (rec, 1)).reduceByKey(lambda Acc, Value: Acc + Value)
wordCountRDD = sc.textFile(sourcePath + "deckofcards.txt").map(lambda rec: rec.split("|")).flatMap(lambda rec: rec).\
map(lambda rec: (rec, 1)).countByKey()
for k, v in wordCountRDD.items(): print("{} : {}".format(k, v))
for i in wordCountReduceByRDD.take(10): print(i)

10 : 4
CLUB : 13
5 : 4
SPADE : 13
3 : 4
2 : 4
BLACK : 26
7 : 4
8 : 4
6 : 4
4 : 4
9 : 4
DIAMOND : 13
K : 4
Q : 4
J : 4
RED : 26
HEART : 13
A : 4
('10', 4)
('4', 4)
('CLUB', 13)
('SPADE', 13)
('J', 4)
('BLACK', 26)
('K', 4)
('RED', 26)
('8', 4)
('9', 4)
